## Обзор данных

Загружаю библиотеку

In [1]:
import pandas as pd

pd.options.mode.chained_assignment = None

Знакомлюсь с данными

In [2]:
df = pd.read_csv('/content/yandex_music_project.csv')
display(df.head())
df.info()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**В итоге**
1. В таблице семь столбцов. Тип данных одинаковый — `object`.
Согласно документации к данным:
* `userID` — идентификатор пользователя
* `Track` — название трека
* `artist` — имя исполнителя
* `genre` — название жанра
* `City` — город пользователя
* `time` — время начала прослушивания
* `Day` — день недели

2. В названиях колонок видны нарушения стиля:
* Строчные буквы сочетаются с прописными
* Встречаются пробелы

3. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

## Предобработка данных


Привожу название колонок к единому виду

In [3]:
df = df.rename(columns = {'  userID' : 'user_id', 'Track' : 'track', '  City  ' : 'city', 'Day' : 'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Ищу пропуски

In [4]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски есть.

Заполняю их 'unknown':
1. В столбцах `track` и `artist` они не повлияют на проверку гипотез.
2. В столбце `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Заполню их и оценю на сколько заполение повлияет на проверку гипотез.

In [5]:
columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Работа с дубликатами


In [6]:
df.duplicated().sum()



3826

Нашлось почти 4000. Удаляю их

In [7]:
df = df.drop_duplicates()

df.duplicated().sum()

0

Проверяю наличие неявных дубликатов в колонке `genre`

In [8]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Неявные дубликаты нашлись. Заменяю их

In [9]:
duplicates = ['hip', 'hop', 'hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, name)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61253 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  61253 non-null  object
 1   track    61253 non-null  object
 2   artist   61253 non-null  object
 3   genre    61253 non-null  object
 4   city     61253 non-null  object
 5   time     61253 non-null  object
 6   day      61253 non-null  object
dtypes: object(7)
memory usage: 3.7+ MB


**В итоге:**

1. Исправлены заголовки.
2. Заполнены пропущенные значения.
3. Удалены явные и неявные дубликаты.



## Проверка гипотез

<u>Проверяю первую гипотезу:</u>
пользователи по-разному слушают музыку в Москве и Санкт-Петербурге.

Для этого беру данные о трёх днях недели — понедельнике, среде и пятнице:
* Группирую пользователей по городам.
* Сравниваю, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [11]:
df.groupby('city')['time'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

В Москве прослушиваний больше, чем в Санкт-Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Группирую данные по дням недели, чтобы получить количество прослушиваний

In [12]:
df.groupby('day')['time'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

Пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создаю функцию `number_tracks()`: она посчитает прослушивания для заданного дня и города с двумя параметрами:
* день недели,
* название города.

In [13]:
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Данные для каждого города в каждый из трёх дней

In [14]:
print('Понедельник, Москва: ', number_tracks('Monday', 'Moscow'))
print('Понедельник, Санкт-Петербург: ', number_tracks('Monday', 'Saint-Petersburg'))
print('Среда, Москва: ', number_tracks('Wednesday', 'Moscow'))
print('Среда, Санкт-Петербург: ', number_tracks('Wednesday', 'Saint-Petersburg'))
print('Пятница, Москва: ', number_tracks('Friday', 'Moscow'))
print('Пятница, Санкт-Петербург: ', number_tracks('Friday', 'Saint-Petersburg'))

Понедельник, Москва:  15740
Понедельник, Санкт-Петербург:  5614
Среда, Москва:  11056
Среда, Санкт-Петербург:  7003
Пятница, Москва:  15945
Пятница, Санкт-Петербург:  5895


Для удобства чтения обернём данные в таблицу:

In [15]:
data = [['Moscow', 15740, 11056, 15945],
        ['Saint-Petersburg', 5614, 7003, 5895]]
column = ['city', 'monday', 'wednesday', 'friday']

info = pd.DataFrame(data = data, columns = column).set_index('city')
display(info)

,monday,wednesday,friday
city,,,
Moscow,15740,11056,15945
Saint-Petersburg,5614,7003,5895


**В итоге:** данные подтвреждают гипотезу.

Данные показывают разницу поведения пользователей:
- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Санкт-Петербурге, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

<u>Проверяю вторую гипотезу:</u>
 утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Создаю две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [16]:
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

Создаю функцию `genre_weekday()` с четырьмя параметрами:
* датасет с данными;
* день недели;
* начальная временная метка в формате 'hh:mm';
* последняя временная метка в формате 'hh:mm'.

Функция будет возвращать топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени

In [17]:
def genre_weekday(df, day, time1, time2):
    genre_df = df[(df['day'] == day)]
    genre_df = genre_df[(genre_df['time'] < time2)]
    genre_df = genre_df[(genre_df['time'] > time1)]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending = False)
    return genre_df_sorted[:10]


Cравниваю результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [18]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [19]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [20]:
 genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [21]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**В итоге:**  данные подтверждают гипотезу частично.
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Стоит отметить, что пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.


Если сравнить топ-10 жанров в понедельник утром, можно сделать выводы:
1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.
2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.
3. Вечер пятницы не влияет на данные.

<u>Проверяю третью гипотезу:</u>

Санкт-Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.
А в Москве преобладает поп-музыка.

Группирую данные по Москве по жанру и посчитаем прослушивания треков каждого и соберём топ-10

In [22]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)
display(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Группирую данные по Санкт-Петербургу по жанру и посчитаем прослушивания треков каждого и соберём топ-10

In [23]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)
display(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**В итоге:** гипотеза частично подтвердилась.
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза.
* Рэп одинаково популярен в Москве и Петербурге.

## Выводы

### Исследование данных показало:
* Первая гипотеза полностью подтвердилась. День недели по-разному влияет на активность пользователей в Москве и Петербурге.
* Вторая гипотеза частично подтверждена. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург.
* Третья гипотеза частично подтверждена. Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Рэп одинаково популярен в Москве и Петербурге.

